## Import Code Dependencies

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!pip install pyod
!pip install shap
!pip install sklearn

     |████████████████████████████████| 113 kB 4.3 MB/s 
  Created wheel for pyod: filename=pyod-0.9.5-py3-none-any.whl size=132699 sha256=c64eefe18a1936caedc7b8af3e4ab591db152b8e3fb47188f4d23e4617880ee6
  Stored in directory: /root/.cache/pip/wheels/3d/bb/b7/62b60fb451b33b0df1ab8006697fba7a6a49709a629055cf77
Successfully built pyod
     |████████████████████████████████| 564 kB 4.1 MB/s 


In [ ]:
import pandas as pd
import shap
from joblib import dump, load
from sklearn.preprocessing import OneHotEncoder as ohe

## Import Data

In [ ]:
df = pd.read_csv("/content/gdrive/MyDrive/Data/featureData.csv")

In [ ]:
df

,claim_id,provider_type,patient_id,billable_start,billable_end,bill_time_difference,provider_name,insurance_company,num_sequence,total_payment,encounter_des,net_value,service,total_visits,claims_per_provider
0,1318172,institutional,1316471,1480936219,1486303819,5367600,None,No Insurance,0,None,None,None,None,2,2730
1,1318174,institutional,1316472,1483032438,1488317238,5284800,None,No Insurance,0,None,None,None,None,2,2730
2,1318176,institutional,1316473,1482231326,1487598926,5367600,None,No Insurance,0,None,None,None,None,1,2730
3,1318178,institutional,1316474,1481408561,1486776161,5367600,None,No Insurance,0,None,None,None,None,2,2730
4,1318180,institutional,1316475,1482495816,1487867016,5371200,None,No Insurance,0,None,None,None,None,2,2730
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12408,c3caa5c5-c5f5-4365-bb22-77bedd27ac1e,institutional,d9b27566-1feb-42dc-88ca-de4f8759a86c,1409396910,1409397810,900,PCP45216,Medicare,2,145.85,General examination of patient (procedure),158.68,"Influenza, seasonal, injectable, preservative ...",21,27
12409,1049ea21-6222-4959-9ad3-15530112e6fa,pharmacy,d9b27566-1feb-42dc-88ca-de4f8759a86c,1465988910,1465992510,3600,COMMUNITY HOSPITAL OF HUNTINGTON PARK,Medicare,1,297.53,Encounter for problem (procedure),None,None,21,65
12410,ea81d0ac-02be-4289-9630-e5cc70c2ac1b,pharmacy,d9b27566-1feb-42dc-88ca-de4f8759a86c,1497092910,1497179310,86400,COMMUNITY HOSPITAL OF HUNTINGTON PARK,Medicare,1,297.53,Inpatient stay (finding),None,None,21,65
12411,a0fb0a6b-b9d1-42de-b9d3-23e3de47e1ef,institutional,34fde5b3-8372-485e-b385-25420146fb54,1521681907,1521682807,900,COMMUNITY HOSPITAL OF HUNTINGTON PARK,Medicare,1,145.85,Telemedicine consultation with patient,None,None,13,65


### Encoding

In [ ]:
# df = df.where(pd.notnull(df), None)
# X =  df.applymap(hash)

# X_train = X
# fName = df.columns.to_list()

In [ ]:
df['net_value'] = df['net_value'].replace('None', 0)
df['total_payment'] = df['total_payment'].replace('None', 0)

In [ ]:
df = df[df.patient_id != 'None']
df = df.dropna(axis=0, subset=['patient_id']) #12394

In [ ]:
df.describe(include='all')

,claim_id,provider_type,patient_id,billable_start,billable_end,bill_time_difference,provider_name,insurance_company,num_sequence,total_payment,encounter_des,net_value,service,total_visits,claims_per_provider
count,12394,12394,12394,1.239400e+04,1.239400e+04,1.239400e+04,12394,12394,12394.000000,12394,12394,12394.0,12394,12394.000000,12394.000000
unique,12394,6,2078,NaN,NaN,NaN,475,13,NaN,1464,46,773.0,69,NaN,NaN
top,1598863,institutional,433620,NaN,NaN,NaN,None,No Insurance,NaN,129.16,General examination of patient (procedure),0.0,None,NaN,NaN
freq,1,9522,335,NaN,NaN,NaN,2711,4506,NaN,4076,3703,7696.0,7699,NaN,NaN
mean,NaN,NaN,NaN,1.235783e+09,1.238050e+09,2.266212e+06,NaN,NaN,1.752299,NaN,NaN,NaN,NaN,67.544941,676.104163
std,NaN,NaN,NaN,4.370709e+08,4.360304e+08,4.271101e+07,NaN,NaN,2.065254,NaN,NaN,NaN,NaN,86.994228,1091.074118
min,NaN,NaN,NaN,-1.451813e+09,-1.451812e+09,0.000000e+00,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,1.000000,1.000000
25%,NaN,NaN,NaN,1.190802e+09,1.194193e+09,9.000000e+02,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,7.000000,27.000000
50%,NaN,NaN,NaN,1.405582e+09,1.405890e+09,9.000000e+02,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,21.000000,109.000000
75%,NaN,NaN,NaN,1.492165e+09,1.496881e+09,5.400000e+03,NaN,NaN,2.000000,NaN,NaN,NaN,NaN,108.000000,400.000000


## column choosing

In [ ]:
columnnames = ['provider_type', 'billable_start', 'billable_end', 'bill_time_difference', 'insurance_company', 'num_sequence', 'total_payment', 'encounter_des', 'net_value', 'service', 'total_visits', 'claims_per_provider']

In [ ]:
claim=df
data = claim[columnnames]

In [ ]:
data.describe(include='all')

,provider_type,billable_start,billable_end,bill_time_difference,insurance_company,num_sequence,total_payment,encounter_des,net_value,service,total_visits,claims_per_provider
count,12394,1.239400e+04,1.239400e+04,1.239400e+04,12394,12394.000000,12394,12394,12394.0,12394,12394.000000,12394.000000
unique,6,NaN,NaN,NaN,13,NaN,1464,46,773.0,69,NaN,NaN
top,institutional,NaN,NaN,NaN,No Insurance,NaN,129.16,General examination of patient (procedure),0.0,None,NaN,NaN
freq,9522,NaN,NaN,NaN,4506,NaN,4076,3703,7696.0,7699,NaN,NaN
mean,NaN,1.235783e+09,1.238050e+09,2.266212e+06,NaN,1.752299,NaN,NaN,NaN,NaN,67.544941,676.104163
std,NaN,4.370709e+08,4.360304e+08,4.271101e+07,NaN,2.065254,NaN,NaN,NaN,NaN,86.994228,1091.074118
min,NaN,-1.451813e+09,-1.451812e+09,0.000000e+00,NaN,0.000000,NaN,NaN,NaN,NaN,1.000000,1.000000
25%,NaN,1.190802e+09,1.194193e+09,9.000000e+02,NaN,1.000000,NaN,NaN,NaN,NaN,7.000000,27.000000
50%,NaN,1.405582e+09,1.405890e+09,9.000000e+02,NaN,1.000000,NaN,NaN,NaN,NaN,21.000000,109.000000
75%,NaN,1.492165e+09,1.496881e+09,5.400000e+03,NaN,2.000000,NaN,NaN,NaN,NaN,108.000000,400.000000


## Installation

In [ ]:
!pip install pyod
!pip install shap
!pip install sklearn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow import keras

from __future__ import division
from __future__ import print_function

## One-hot Encoding

In [ ]:
import shap
from sklearn.preprocessing import OneHotEncoder as ohe

In [ ]:
encoding_col = ['provider_type', 'insurance_company','service']
#encounter_des

In [ ]:
encoding_data = data[encoding_col]
df1 = encoding_data

In [ ]:
encoding_data = pd.get_dummies(df1,prefix=encoding_col)

In [ ]:
non_encoding = data[['billable_start', 'billable_end', 'bill_time_difference', 'num_sequence', 'total_payment', 'net_value']]

In [ ]:
datafinal = pd.concat([encoding_data, non_encoding],axis=1)

In [ ]:
datafinal

,provider_type_None,provider_type_institutional,provider_type_oral,provider_type_pharmacy,provider_type_professional,provider_type_vision,insurance_company_Aetna,insurance_company_Anthem,insurance_company_Blue Cross Blue Shield,insurance_company_Cigna Health,insurance_company_Dual Eligible,insurance_company_Humana,insurance_company_Kela,insurance_company_Medicaid,insurance_company_Medicare,insurance_company_No Insurance,insurance_company_UnitedHealthcare,insurance_company_medicare,insurance_company_private,service_Allergy screening test,service_Antenatal RhD antibody screening,service_Augmentation of labor,service_Bilateral tubal ligation,service_Biopsy of prostate,service_Cardiac MRI,service_Catheter ablation of tissue of heart,service_Childbirth,service_Colonoscopy,service_Computed tomography of chest and abdomen,service_Counseling for termination of pregnancy,service_DTaP,service_Digital examination of rectum,service_Documentation of current medications,service_Echocardiography (procedure),service_Electrical cardioversion,service_Epidural anesthesia,service_Episiotomy,service_Evaluation of uterine fundal height,service_Excision of fallopian tube and surgical removal of ectopic pregnancy,service_Fetal anatomy study,...,service_Lumpectomy of breast (procedure),service_Mammography (procedure),service_Manual pelvic examination (procedure),service_Medical induction of labor,service_Medication Reconciliation (procedure),service_Methotrexate injection into tubal pregnancy,service_None,service_Peripheral blood smear interpretation,service_Physical examination,service_Physical examination following birth,service_Plain chest X-ray (procedure),service_Pneumococcal conjugate PCV 13,service_Pregnancy termination care,service_Radiation therapy care (regime/therapy),service_Referral to hypertension clinic,service_Removal of intrauterine device,service_Removal of subcutaneous contraceptive,service_Renal dialysis (procedure),service_Replacement of contraceptive intrauterine device,service_Replacement of subcutaneous contraceptive,service_RhD passive immunization,service_Seizure Count Cerebral Cortex Electroencephalogram (EEG),service_Spirometry (procedure),service_Streptococcus pneumoniae group B antigen test,service_Subcutaneous immunotherapy,service_Surgical manipulation of joint of knee,service_Td (adult) preservative free,service_Tdap,service_Transthoracic three dimensional ultrasonography of heart (procedure),service_meningococcal MCV4P,"service_pneumococcal polysaccharide vaccine, 23 valent",service_trans thoracic echocardiogram,service_varicella,service_zoster,billable_start,billable_end,bill_time_difference,num_sequence,total_payment,net_value
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1480936219,1486303819,5367600,0,0,0
1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1483032438,1488317238,5284800,0,0,0
2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1482231326,1487598926,5367600,0,0,0
3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1481408561,1486776161,5367600,0,0,0
4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1482495816,1487867016,5371200,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12408,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,

In [ ]:
X_train = datafinal

## Model Training

### Isolation Forest

In [ ]:
from pyod.models.iforest import IForest

# train IForest detector
isofor = IForest() #default contamination is 0.1
isofor = isofor.fit(X_train)
dump(isofor, "/content/gdrive/MyDrive/Data/Trained_Models/IsolatedForest.joblib")

['/content/gdrive/MyDrive/Data/Trained_Models/IsolatedForest.joblib']

### Auto Encoder

In [ ]:
from pyod.models.auto_encoder import AutoEncoder

autenc = AutoEncoder(epochs=15, hidden_neurons =[15, 14, 14, 15], contamination = 0.1)
autenc = autenc.fit(X_train)
dump(autenc, "/content/gdrive/MyDrive/Data/Trained_Models/AutoEncoder.joblib")

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 94)                8930      
                                                                 
 dropout (Dropout)           (None, 94)                0         
                                                                 
 dense_1 (Dense)             (None, 94)                8930      
                                                                 
 dropout_1 (Dropout)         (None, 94)                0         
                                                                 
 dense_2 (Dense)             (None, 15)                1425      
                                                                 
 dropout_2 (Dropout)         (None, 15)                0         
                                                                 
 dense_3 (Dense)             (None, 14)                2

['/content/gdrive/MyDrive/Data/Trained_Models/AutoEncoder.joblib']

### PCA

In [ ]:
from pyod.models.pca import PCA
pca = PCA()
pca = pca.fit(X_train)
dump(pca, "/content/gdrive/MyDrive/Data/Trained_Models/PCA.joblib")

['/content/gdrive/MyDrive/Data/Trained_Models/PCA.joblib']

### visualization - PCA

In [ ]:
datafinal_summary = shap.kmeans(datafinal, 10)

In [ ]:
explainerPCA = shap.KernelExplainer(pca.predict, datafinal_summary )

In [ ]:
shap_values_PCA = explainerPCA.shap_values(datafinal)

  0%|          | 0/12394 [00:00<?, ?it/s]

In [ ]:
pd_shap_PCA = pd.DataFrame(shap_values_PCA)

In [ ]:
pd_shap_PCA 

In [ ]:
shap_values_PCA_save = pd_shap_PCA .to_csv('/content/gdrive/MyDrive/PCA_shap.csv')

In [ ]:
shap_values_PCA.shape

In [ ]:
shap_values_PCA[1:5,:].shape

In [ ]:
shap.summary_plot(shap_values_PCA, datafinal)

In [ ]:
s1 = shap_values_PCA[:,1:20]
d1 = datafinal.iloc[:,1:20]
shap.summary_plot(s1,d1)

In [ ]:
s2 = shap_values_PCA[:,21:40]
d2 = datafinal.iloc[:,21:40]
shap.summary_plot(s2,d2)

In [ ]:
s3 = shap_values_PCA[:,41:60]
d3 = datafinal.iloc[:,41:60]
shap.summary_plot(s3,d3)

In [ ]:
s4 = shap_values_PCA[:,61:80]
d4 = datafinal.iloc[:,61:80]
shap.summary_plot(s4,d4)

In [ ]:
s5 = shap_values_PCA[:,81:93]
d5 = datafinal.iloc[:,81:93]
shap.summary_plot(s5,d5)

In [ ]:
shap.summary_plot(shap_values_PCA, datafinal, plot_type="bar")

In [ ]:
#single claim
shap.initjs()
shap.force_plot(explainerPCA.expected_value, shap_values_PCA[10000]) # feature_names= col_names

### visualization - PCA(Non-encoding part)

In [ ]:
explainer PCA

## Result Explanation

In [ ]:
# explain the model's predictions using SHAP
# datafinal_summary = shap.kmeans(X_train, 10)
# explainer = shap.KernelExplainer(isofor.predict,datafinal_summary)
# shap_values = explainer.shap_values(X_train)

explainer = shap.Explainer(isofor)
# shap_values = explainer(X_train)

In [ ]:
# Runs prediction of the *record* on PCA, Isolation Forest, and AutoEncoder
# Returns expectation of fraud and chart demonstrating features impacting decision
def prediction(record):
    p = pca.predict(record)
    i = isofor.predict(record)
    a = autenc.predict(record)   
    sum = p+i+a
    if sum == 3:
        print("High Chance of Fraud")
    elif sum == 2:
        print("Medium Chance of Fraud")
    elif sum == 1:
         print("Low Chance of Fraud")
    elif sum == 0:
         print("Unlikely to be fraud")
    if sum > 0:
        print("Isolation Forest result: ", i , ", PCA result: " , p , ", and Auto Encoder result: ", a)
        print("The features in red have the highest impact on this record being fraudulent")
        
        shap_values = explainer(X_train)
        shap.initjs()
        print(shap_values.values)
        beeswarm_plot = shap.plots.beeswarm(shap_values, max_display=X_train.shape[1])
        force_plot = shap.force_plot(explainer.expected_value, shap_values.values[12344], record, feature_names = fName, matplotlib=True, contribution_threshold = 0.075)
        summary_plot = shap.summary_plot(shap_values, X_train, show=False, max_display=X_train.shape[1], plot_type="bar")
        #shap.plots.text(shap_values[2]
        

In [ ]:
def read_txt_file(file_path):
    with open(file_path,'r') as f:
        content = [item.strip() for item in f.readlines()]
        features = []
        for line in content:
            parts = line.split('\t')
            features.append(parts)
        return features

In [ ]:
#last row of record
# for i in X_train.index:
record = X_train.iloc[12391]
print(record)
prediction([record])

In [ ]:
labels = pd.DataFrame({"label_pca": pca.labels_, "label_ae":autenc.labels_, "label_if": isofor.labels_})
labels['sum']= labels.sum(axis=1)
print(labels)
print(labels.loc[labels['sum'] == 3].index)
print(labels.loc[labels['sum'] == 2].index)
print(labels.loc[labels['sum'] == 1].index)
print(labels.loc[labels['sum'] == 0].index)
print(labels['sum'].value_counts())
print(labels.value_counts())